In [1]:
#import the necessary libraries
import tensorflow as tf
import numpy as np
import os
import matplotlib.pyplot as plt
import cv2 as cv
import librosa

In [2]:
#specify the paths for your audio files
LJSpeech = r"C:\Users\ANING\Downloads\LJSpeech300"
CVcorpus = r"C:\Users\ANING\Downloads\CVcorpus"

In [3]:
#specify the folder to store your various spectrum images
Spectr_image = r"C:\Users\ANING\Downloads\Automatic-Voice-Recognition-main\Spectrum_images"

In [ ]:
#create folders for the different audio files
os.makedirs('Spectrum_images\CVcorpus')
os.makedirs('Spectrum_images\LJSpeech')

In [5]:

#function to process audio and generate the spectrum images
def process_audio(audio_path,save_location,filename):   
    audio, fs = librosa.load(audio_path)
    D = librosa.amplitude_to_db(librosa.stft(audio), ref=np.max)
    # Save the spectrogram
    plt.figure(figsize=(10, 5))
    plt.axis('off')
    librosa.display.specshow(D, sr=fs, x_axis='time', y_axis='linear',cmap='viridis')
    plt.savefig(save_location+'\\'+filename+'.jpg', dpi=300, bbox_inches='tight', pad_inches=0,transparent=True)


In [6]:
#function 
def crop_image(image):
    imag = np.array(image)
    # print(imag)
    mask = np.zeros(image.shape,dtype=np.uint8)
    cv.rectangle(mask,(100,50),(710,585),(255,255,255),-1)
    image = np.bitwise_and(imag,mask)
    x, y, w, h = 100,50,610,535

    # Crop the image using the bounding rectangle coordinates
    cropped_image = image[y:y + h, x:x + w]
    image =np.array(image)
    return cropped_image

In [ ]:
#process the audio files to generate the spectrum images
for audios in os.listdir(CVcorpus):
    process_audio(CVcorpus+f'\\{audios}',r"C:\Users\ANING\Downloads\Automatic-Voice-Recognition-main\Spectrum_images\CVcorpus",audios)

In [ ]:
#process the audio files to generate the spectrum images
for audios in os.listdir(LJSpeech):
    process_audio(LJSpeech+f'\\{audios}',r'C:\Users\ANING\Downloads\Automatic-Voice-Recognition-main\Spectrum_images\LJSpeech',audios)

In [8]:
image_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255)

In [9]:
train_images = image_datagen.flow_from_directory(Spectr_image,target_size=(300,300),class_mode='binary')

Found 601 images belonging to 2 classes.


In [10]:
model = tf.keras.models.Sequential()
model.add(tf.keras.applications.vgg16.VGG16(include_top=False,weights='imagenet',input_shape=(300,300,3)))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(512,activation='relu'))
model.add(tf.keras.layers.Dense(32,activation='relu'))
model.add(tf.keras.layers.Dense(1,activation='sigmoid'))

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 9, 9, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 41472)             0         
                                                                 
 dense (Dense)               (None, 512)               21234176  
                                                                 
 dense_1 (Dense)             (None, 1)                 513       
                                                                 
Total params: 35,949,377
Trainable params: 35,949,377
Non-trainable params: 0
_________________________________________________________________


In [12]:
model.compile(optimizer='sgd',loss='binary_crossentropy',metrics=['accuracy'])

In [13]:
model.fit(train_images,epochs=10,)

Epoch 1/10
19/19 [==============================] - 429s 23s/step - loss: 0.9572 - accuracy: 0.6988
Epoch 2/10
19/19 [==============================] - 606s 32s/step - loss: 0.3365 - accuracy: 0.8819
Epoch 3/10
19/19 [==============================] - 496s 26s/step - loss: 0.1587 - accuracy: 0.9468
Epoch 4/10
19/19 [==============================] - 486s 26s/step - loss: 0.0487 - accuracy: 0.9917
Epoch 5/10
19/19 [==============================] - 436s 23s/step - loss: 0.0224 - accuracy: 0.9933
Epoch 6/10
19/19 [==============================] - 455s 24s/step - loss: 0.0112 - accuracy: 1.0000
Epoch 7/10
19/19 [==============================] - 519s 28s/step - loss: 0.0060 - accuracy: 1.0000
Epoch 8/10
19/19 [==============================] - 457s 24s/step - loss: 0.0041 - accuracy: 1.0000
Epoch 9/10
19/19 [==============================] - 453s 24s/step - loss: 0.0035 - accuracy: 1.0000
Epoch 10/10
19/19 [==============================] - 454s 24s/step - loss: 0.0025 - accuracy: 1.0000

In [14]:
model.save('final_model.h5')